<a href="https://colab.research.google.com/github/NicoWeiner/MasterProject/blob/master/Immobilienscout24_Text_Classification_%26_LIWC_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Immobilienscout24 - Text Classification & LIWC Analysis
---



## 1.  Importing packages & the Immobilienscout24 scraped data 

In [0]:
# !pip install pandas 
# !pip install gensim
# !pip install spacy
# !pip install gensim
# !pip install matplotlib
# !pip install sklearn
# !pip install keras
# !pip install tensorflow
# !pip install scikit-learn
# !pip install scipy
# !pip install nltk
# !pip install random2
# !pip install warnings-plugin
# !pip install statsmodels
# !pip install seaborn
# !pip install DateTime
# !pip install datetime3

import gensim
import numpy as np
import spacy
from spacy import displacy
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import matplotlib.pyplot as plt
import keras
import tensorflow
import statsmodels.api as sm
import sklearn
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import random
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from pandas import DataFrame, read_csv
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
import os
%matplotlib inline

#For Scikit Regression: 

from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib notebook


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
immodf = pd.read_csv("https://www.dropbox.com/s/04s1ptlkvy5l61s/FINAL_IMMOBILIENSCOUT24_03_05_20.csv?dl=1", sep=',', parse_dates=['Baujahr', 'Bezugsfrei_ab'], error_bad_lines=False, dtype='unicode', lineterminator='\n')
pd.options.display.max_columns = None
immodf.head(5)


,ID,ScoutID,Typ,Kaufpreis,Provision,Energieeffizienzklasse,Objektzustand,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,Bezugsfrei_ab,Title,Beschreibung,Ausstattung,Lagebeschreibung,Sonstiges,PLZ,Stadt,Ortsteil,ScrapingDate1,ScrapingDate2,ScrapingDate3,ScrapingDate4,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Average_income
0,115756801,Objekt-Nr.: FB211 | Scout-ID: 115756801,Bungalow,518500.0,Nein,A+,Erstbezug,3.0,160.0,1200.0,2020,NaN,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,"3 Zimmer, offene Küche, Diele, Bad, Dusche, Ha...","Das Baugrundstück befindet sich in Dauernheim,...",Kern-Haus baut individuelle und massive Archit...,63691,Ranstadt,Wetteraukreis,2020-04-21,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
1,116853306,Objekt-Nr.: 562020 | Scout-ID: 116853306,Einfamilienhaus (freistehend),301667.0,Nein,NaN,Erstbezug,4.0,135.0,860.0,NaN,NaN,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,In dem oben genannten Kaufpreis wurden die Kos...,Die Häuser werden auf realen Grundstücken ange...,Deutschland`s meistverkaufte Ausbauhäuser!\n\n...,52385,Nideggen,Düren,2020-04-21,NaN,NaN,NaN,4,65.04,9945.0,4902.0,5043.0,153.0,NaN
2,116959865,Objekt-Nr.: W16091 | Scout-ID: 116959865,Einfamilienhaus (freistehend),225900.0,Nein,A+,Erstbezug,3.0,76.0,612.0,2021,NaN,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,"3 Zimmer, offene Küche, Diele mit Garderobe, B...",- Grundstück befindet sich in Elstertrebnitz i...,Kern-Haus\r\n\r\nKern-Haus baut individuelle u...,04523,Pegau,Leipzig,2020-04-21,2020-04-25,2020-04-29,NaN,12,48.8,6288.0,3068.0,3220.0,129.0,NaN
3,116953174,Scout-ID: 116953174,Einfamilienhaus (freistehend),218659.0,Nein,NaN,Erstbezug,6.0,144.0,550.0,2020,NaN,Das neue LifeStyle 14.03S ! Auch als Stadtvill...,Jeder von uns hat Träume. Ein eigenes Haus zu ...,ZERTIFIZIERTE QUALITÄT IN ALLEN BEREICHEN!\n\n...,"Ich bin Ihr Ansprechpartner in Ihrer Region, w...",Profitieren Sie vom aktuell niedrigen Zinsnive...,52428,52428,Jülich,2020-04-21,2020-04-25,2020-04-29,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN
4,116953218,Scout-ID: 116953218,Einfamilienhaus (freistehend),287200.0,Nein,NaN,Erstbezug,4.0,141.0,801.0,NaN,NaN,"Barriere freier Bungalow in Kehrigk , jetzt ko...",Dieses Angebot beinhaltet:\n\n- Projektiertes ...,"Ein wahrliches traumhaftes Haus, was ihrer Fam...",Das Grundstück liegt in Kehrigk\n15859 Storkow...,Auch ohne Eigenkapital bauen !! \nProfitieren ...,15859,Storkow,Storkow,2020-04-21,2020-04-25,2020-04-29,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN


## 2.  Run NFM Model to extract common topics in listing descriptions



### 2.1. Removing all German stopwords in the listing descriptions as we only want to extract the relevant words

In [0]:
stopWords = set (stopwords.words('german'))

print(len(stopWords))
print(stopWords)

232
{'welcher', 'kein', 'muss', 'demselben', 'anderm', 'jetzt', 'wir', 'dass', 'dasselbe', 'mir', 'jedem', 'meinem', 'werden', 'keinen', 'seine', 'dazu', 'andern', 'wenn', 'es', 'hatte', 'welchem', 'daß', 'dies', 'bei', 'für', 'nach', 'solchem', 'sein', 'anderen', 'allen', 'anders', 'jener', 'unter', 'nicht', 'sie', 'den', 'jenem', 'bin', 'ihres', 'solches', 'haben', 'zwischen', 'eures', 'würde', 'unsere', 'meine', 'weg', 'einiger', 'hatten', 'vor', 'soll', 'sehr', 'unser', 'jeder', 'des', 'deines', 'vom', 'dort', 'mich', 'andere', 'indem', 'warst', 'sich', 'derer', 'diesen', 'ander', 'ins', 'keinem', 'jedes', 'nichts', 'unseren', 'allem', 'jene', 'sondern', 'hinter', 'zum', 'weil', 'wollte', 'uns', 'dieselben', 'der', 'ihrer', 'im', 'solche', 'solchen', 'ohne', 'sollte', 'aller', 'aber', 'ist', 'mancher', 'bist', 'desselben', 'dieselbe', 'manches', 'meiner', 'wird', 'also', 'diese', 'manchem', 'unseres', 'zu', 'jeden', 'so', 'einigem', 'welches', 'dich', 'einigen', 'oder', 'bis', 'zur

In [0]:
tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words=stopWords, token_pattern='[a-zA-Z][a-zA-Z]{2,}')
doc_term_matrix = tfidf_vect.fit_transform(immodf['Beschreibung'].values.astype('U'))


### 2.2. Creating a probability matrix for topics

In [0]:
nmf = NMF(n_components=25, random_state=42)
nmf.fit(doc_term_matrix)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=25, random_state=42, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [0]:
#retrieve the probability vector of words 

first_topic = nmf.components_[1]
top_topic_words = first_topic.argsort()[-10:]

### 2.3. Extracting top words for 25 topics

In [0]:
#the words in the data grouped by topics

for i,topic in enumerate(nmf.components_):
    print(f'Top words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

Top words for topic #0:
['offene', 'giges', 'gig', 'dusche', 'kamin', 'wohnbereich', 'garten', 'ste', 'zugang', 'terrasse', 'wohn', 'gigen', 'essbereich', 'gige', 'gro']


Top words for topic #1:
['garageninnenhof', 'garagenraum', 'garagenkeller', 'garagenmiete', 'garagennebengeb', 'garagenneubau', 'garagennutzung', 'garagenparkplatz', 'garagenparzelle', 'garagenpl', 'garagenplatz', 'garagenplatzes', 'garagenkomplex', 'zzt', 'nan']


Top words for topic #2:
['wfl', 'wunschimmobilie', 'notarkosten', 'preisg', 'montag', 'nachlass', 'ausgabe', 'immobilienversteigerungs', 'kataloginfos', 'kataloges', 'versteigert', 'eur', 'sichern', 'verkehrswert', 'telefonnummer']


Top words for topic #3:
['mieter', 'insgesamt', 'derzeit', 'eur', 'mieteinnahmen', 'garagen', 'stellpl', 'drei', 'tze', 'jeweils', 'zwei', 'wohneinheiten', 'vermietet', 'mehrfamilienhaus', 'wohnungen']


Top words for topic #4:
['senden', 'interesse', 'eigent', 'pers', 'besichtigungstermin', 'nnen', 'vollst', 'immobilie', 'erh

In [0]:
# Append new column with topics as numerical values
test = immodf

topic_values = nmf.transform(doc_term_matrix)
test['Topic'] = topic_values.argmax(axis=1)
test.head(3)

,ID,ScoutID,Typ,Kaufpreis,Provision,Energieeffizienzklasse,Objektzustand,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,Bezugsfrei_ab,Title,Beschreibung,Ausstattung,Lagebeschreibung,Sonstiges,PLZ,Stadt,Ortsteil,ScrapingDate1,ScrapingDate2,ScrapingDate3,ScrapingDate4,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Average_income,Topic
0,115756801,Objekt-Nr.: FB211 | Scout-ID: 115756801,Bungalow,518500.0,Nein,A+,Erstbezug,3.0,160.0,1200.0,2020,NaN,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,"3 Zimmer, offene Küche, Diele, Bad, Dusche, Ha...","Das Baugrundstück befindet sich in Dauernheim,...",Kern-Haus baut individuelle und massive Archit...,63691,Ranstadt,Wetteraukreis,2020-04-21,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,0
1,116853306,Objekt-Nr.: 562020 | Scout-ID: 116853306,Einfamilienhaus (freistehend),301667.0,Nein,NaN,Erstbezug,4.0,135.0,860.0,NaN,NaN,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,In dem oben genannten Kaufpreis wurden die Kos...,Die Häuser werden auf realen Grundstücken ange...,Deutschland`s meistverkaufte Ausbauhäuser!\n\n...,52385,Nideggen,Düren,2020-04-21,NaN,NaN,NaN,4,65.04,9945.0,4902.0,5043.0,153.0,NaN,12
2,116959865,Objekt-Nr.: W16091 | Scout-ID: 116959865,Einfamilienhaus (freistehend),225900.0,Nein,A+,Erstbezug,3.0,76.0,612.0,2021,NaN,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,"3 Zimmer, offene Küche, Diele mit Garderobe, B...",- Grundstück befindet sich in Elstertrebnitz i...,Kern-Haus\r\n\r\nKern-Haus baut individuelle u...,04523,Pegau,Leipzig,2020-04-21,2020-04-25,2020-04-29,NaN,12,48.8,6288.0,3068.0,3220.0,129.0,NaN,24


### 2.4. Give suitable topics a title 

Considering the output of the 25 topics above, we now select the ones that make sense and are suitable and categorize them with a name. The category name tells what a house description is focusing on. The ones that are do not make sense are later being dropped. 


In [0]:
# Insert the output in dataset as new variables

test.Topic = test.Topic.astype('str')

test.Topic = test.Topic.replace(['1'],'garage')
test.Topic = test.Topic.replace(['2'],'house_auction')
test.Topic = test.Topic.replace(['3', '16'],'multi_dwelling')
test.Topic = test.Topic.replace(['4'],'house_viewing')
test.Topic = test.Topic.replace(['7'],'renovated')
test.Topic = test.Topic.replace(['13'],'house_layout')
test.Topic = test.Topic.replace(['5', '22'],'courtyard_estate')

In [0]:
test = test.astype('str')


In [0]:
test

,ID,ScoutID,Typ,Kaufpreis,Provision,Energieeffizienzklasse,Objektzustand,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,Bezugsfrei_ab,Title,Beschreibung,Ausstattung,Lagebeschreibung,Sonstiges,PLZ,Stadt,Ortsteil,ScrapingDate1,ScrapingDate2,ScrapingDate3,ScrapingDate4,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Average_income,Topic
0,115756801,Objekt-Nr.: FB211 | Scout-ID: 115756801,Bungalow,518500.0,Nein,A+,Erstbezug,3.0,160.0,1200.0,2020,nan,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,"3 Zimmer, offene Küche, Diele, Bad, Dusche, Ha...","Das Baugrundstück befindet sich in Dauernheim,...",Kern-Haus baut individuelle und massive Archit...,63691,Ranstadt,Wetteraukreis,2020-04-21,nan,nan,nan,4,nan,nan,nan,nan,nan,nan,0
1,116853306,Objekt-Nr.: 562020 | Scout-ID: 116853306,Einfamilienhaus (freistehend),301667.0,Nein,nan,Erstbezug,4.0,135.0,860.0,nan,nan,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,In dem oben genannten Kaufpreis wurden die Kos...,Die Häuser werden auf realen Grundstücken ange...,Deutschland`s meistverkaufte Ausbauhäuser!\n\n...,52385,Nideggen,Düren,2020-04-21,nan,nan,nan,4,65.04,9945.0,4902.0,5043.0,153.0,nan,12
2,116959865,Objekt-Nr.: W16091 | Scout-ID: 116959865,Einfamilienhaus (freistehend),225900.0,Nein,A+,Erstbezug,3.0,76.0,612.0,2021,nan,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,"3 Zimmer, offene Küche, Diele mit Garderobe, B...",- Grundstück befindet sich in Elstertrebnitz i...,Kern-Haus\r\n\r\nKern-Haus baut individuelle u...,04523,Pegau,Leipzig,2020-04-21,2020-04-25,2020-04-29,nan,12,48.8,6288.0,3068.0,3220.0,129.0,nan,24
3,116953174,Scout-ID: 116953174,Einfamilienhaus (freistehend),218659.0,Nein,nan,Erstbezug,6.0,144.0,550.0,2020,nan,Das neue LifeStyle 14.03S ! Auch als Stadtvill...,Jeder von uns hat Träume. Ein eigenes Haus zu ...,ZERTIFIZIERTE QUALITÄT IN ALLEN BEREICHEN!\n\n...,"Ich bin Ihr Ansprechpartner in Ihrer Region, w...",Profitieren Sie vom aktuell niedrigen Zinsnive...,52428,52428,Jülich,2020-04-21,2020-04-25,2020-04-29,nan,12,nan,nan,nan,nan,nan,nan,10
4,116953218,Scout-ID: 116953218,Einfamilienhaus (freistehend),287200.0,Nein,nan,Erstbezug,4.0,141.0,801.0,nan,nan,"Barriere freier Bungalow in Kehrigk , jetzt ko...",Dieses Angebot beinhaltet:\n\n- Projektiertes ...,"Ein wahrliches traumhaftes Haus, was ihrer Fam...",Das Grundstück liegt in Kehrigk\n15859 Storkow...,Auch ohne Eigenkapital bauen !! \nProfitieren ...,15859,Storkow,Storkow,2020-04-21,2020-04-25,2020-04-29,nan,12,nan,nan,nan,nan,nan,nan,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48877,37000048,Scout-ID: 37000048,Besondere Immobilie,239000.0,Nein,nan,Modernisiert,10.0,212.0,646.0,nan,nan,Mehrfamilienhaus im Wintersportzentrum Oberhof...,Erdgeschoß: 2 Zimmer + Küche + Bad\r\nhochwert...,Erdgeschoss: mit Einbauküche und Kamin und Zug...,Stadtzentrum/Fußgängerzone\r\nEntfernung zum B...,Bei Interesse kontaktieren Sie mich bitte tele...,98559,Oberhof,Schmalkalden-Meiningen,2020-04-21,2020-04-25,2020-04-29,nan,12,23.4,1608.0,788.0,820.0,69.0,nan,18
48878,36788963,Scout-ID: 36788963,Doppelhaushälfte,188500.0,Nein,nan,Erstbezug,6.0,164.0,339.0,nan,2020,Spitzenlage - unverbaubaurer Weitblick - Provi...,Große Doppelhaushälfte mit noch frei zu gestal...,Sehr umfangreiche Ausstattung und hochwertige ...,"Unverbaubare Aussicht über die Stadt, weite Fe...",Die Heizungstechnik ist noch frei wählbar.\nDe...,55606,Kirn,Bad Kreuznach,2020-04-21,2020-04-25,2020-04-29,nan,12,16.53,8193.0,3987.0,4206.0,496.0,nan,8
48879,36713355,Objekt-Nr.: 1477028 | Scout-ID: 36713355,Einfamilienhaus (freistehend),169000.0,Nein,nan,Neuwertig,3.0,96.0,380.0,2002,kurzfristig,Heidekrug - wunderschöner freistehender Bungalow,"Unser schöner, freistehender Bungalow ist sofo...",nan,nan,Besichtigungen si

## 3. Import results of LIWC analysis & merge with Immoscout data

Outside of this notebook, the program LIWC2015 has been purchased which allowed to run multi-level linguistic analysis on the listing descriptions. The analysis has been conducted within the program LIWC2015, resulting in a new dataset with new columns that indicate the level of various linguistic features and sentiment. For more details on the particular linguistic feature, see: https://liwc.wpengine.com/compare-dictionaries/

In [0]:
liwc1 = pd.read_csv("https://www.dropbox.com/s/18rn8yb2kmnivjy/LIWC2015%20Results%202%20%28FINAL_IMMOBILIENSCOUT24_03_05_20.csv%29.csv?dl=1", sep=',', error_bad_lines=False, dtype='unicode', lineterminator='\n')
pd.options.display.max_columns = None
liwc1 = liwc1.drop(labels=0, axis=0)
liwc1

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,AA,AB,AC,AD,AE,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you_total,ipron,article,prep,auxverb,adverb,conj,negate,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achiev,power,reward,risk,focuspast,focuspresent,focusfuture,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu\r
1,115756801,Objekt-Nr.: FB211 | Scout-ID: 115756801,Bungalow,518500.0,Nein,A+,Erstbezug,3.0,160.0,1200.0,2020,NaN,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,"3 Zimmer, offene Küche, Diele, Bad, Dusche, Ha...","Das Baugrundstück befindet sich in Dauernheim,...",Kern-Haus baut individuelle und massive Archit...,63691,Ranstadt,Wetteraukreis,2020-04-21,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,260,99.00,75.31,33.30,99.00,15.29,32.31,79.23,45.00,8.46,3.46,0.00,0.00,0.38,3.46,14.23,15.00,6.54,4.62,10.77,0.00,6.15,6.15,0.00,0.00,0.00,0.00,10.38,0.00,1.15,0.00,0.00,13.08,2.31,1.15,1.15,2.69,1.54,3.46,2.31,1.15,0.77,0.38,4.23,0.38,0.77,0.00,1.92,8.08,1.15,2.69,2.31,2.31,0.00,0.38,3.46,1.92,4.23,5.38,6.54,0.38,0.38,0.00,1.15,0.00,0.38,0.00,0.00\r
2,116853306,Objekt-Nr.: 562020 | Scout-ID: 116853306,Einfamilienhaus (freistehend),301667.0,Nein,NaN,Erstbezug,4.0,135.0,860.0,NaN,NaN,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,In dem oben genannten Kaufpreis wurden die Kos...,Die Häuser werden auf realen Grundstücken ange...,Deutschland`s meistverkaufte Ausbauhäuser!\n\n...,52385,Nideggen,Düren,2020-04-21,NaN,NaN,NaN,4,65.04,9945.0,4902.0,5043.0,153.0,NaN,162,46.46,80.34,24.22,89.37,13.50,35.19,75.93,48.15,16.05,11.11,2.47,0.62,0.00,3.09,6.79,13.58,7.41,4.32,15.43,1.85,3.09,3.09,0.00,0.00,0.00,0.00,18.52,1.23,0.62,0.00,0.00,10.49,2.47,0.62,0.00,1.85,2.47,3.09,3.09,0.62,1.23,0.00,1.85,0.00,0.62,0.00,0.62,8.02,3.09,4.32,0.00,1.23,0.00,1.23,4.32,1.85,3.70,2.47,3.70,1.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r
3,116959865,Objekt-Nr.: W16091 | Scout-ID: 116959865,Einfamilienhaus (freistehend),225900.0,Nein,A+,Erstbezug,3.0,76.0,612.0,2021,NaN,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,"3 Zimmer, offene Küche, Diele mit Garderobe, B...",- Grundstück befindet sich in Elstertrebnitz i...,Kern-Haus\r\n\r\nKern-Haus baut individuelle u...,04523,Pegau,Leipzig,2020-04-21,2020-04-25,2020-04-29,NaN,12,48.8,6288.0,3068.0,3220.0,129.0,NaN,94,95.21,77.64,17.77,99.00,11.75,28.72,77.66,43.62,15.96,4.26,0.00,0.00,1.06,7.45,10.64,14.89,2.13,1.06,13.83,2.13,7.45,7.45,0.00,0.00,0.00,0.00,14.89,1.06,2.13,0.00,0.00,11.70,1.06,0.00,1.06,4.26,1.06,4.26,3.19,2.13,1.06,0.00,1.06,0.00,1.06,0.00,0.00,5.32,1.06,2.13,0.00,2.13,1.06,0.00,3.19,1.06,3.19,1.06,6.38,0.00,0.00,0.00,1.06,0.00,1.06,0.00,0.00\r
4,116953174,Scout-ID: 116953174,Einfamilienhaus (freistehend),218659.0,Nein,NaN,Erstbezug,6.0,144.0,550.0,2020,NaN,Das neue LifeStyle 14.03S ! Auch als Stadtvill...,Jeder von uns hat Träume. Ein eigenes Haus zu ...,ZERTIFIZIERTE QUALITÄT IN ALLEN BEREICHEN!\n\n...,"Ich bin Ihr Ansprechpartner in Ihrer Region, w...",Profitieren Sie vom aktuell niedrigen Zinsnive...,52428,52428,Jülich,2020-04-21,2020-04-25,2020-04-29,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,148,93.60,95.48,2.04,99.00,13.45,37.16,76.35,45.95,13.51,9.46,0.00,4.05,2.03,3.38,8.11,15.54,7.43,4.05,9.46,0.00,6.76,6.08,0.00,0.00,0.00,0.00,12.84,0.68,0.68,0.00,0.68,10.14,0.68,1.35,0.68,0.68,2.03,0.68,0.68,0.00,0.00,0.68,4.05,0.68,0.68,1.35,0.00,13.51,5.41,6.76,0.68,0.00,0.00,0.00,5.41,2.03,4.73,2.03,3.38,0.68,0.00,0.00,2.03,0.00,0.00,0.68,0.00\r
5,116953218,Scout-ID: 116953218,Einfamilienhaus (freistehend),287200.0,Nein,NaN,Erstbezug,4.0,141.0,801.0,NaN,NaN,"Barriere freier Bungalow in Kehrigk , jetzt ko...",Dieses Angebot beinhaltet:\n\n- Projektiertes ...,"Ein wahrli

In [0]:
liwc2 = liwc1
liwc2 = liwc2.drop(['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'AA', 'AB', 'AC', 'AD', 'AE'], axis=1) 
liwc2 = liwc2.astype('str')
liwc2


,A,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you_total,ipron,article,prep,auxverb,adverb,conj,negate,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achiev,power,reward,risk,focuspast,focuspresent,focusfuture,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu\r
1,115756801,260,99.00,75.31,33.30,99.00,15.29,32.31,79.23,45.00,8.46,3.46,0.00,0.00,0.38,3.46,14.23,15.00,6.54,4.62,10.77,0.00,6.15,6.15,0.00,0.00,0.00,0.00,10.38,0.00,1.15,0.00,0.00,13.08,2.31,1.15,1.15,2.69,1.54,3.46,2.31,1.15,0.77,0.38,4.23,0.38,0.77,0.00,1.92,8.08,1.15,2.69,2.31,2.31,0.00,0.38,3.46,1.92,4.23,5.38,6.54,0.38,0.38,0.00,1.15,0.00,0.38,0.00,0.00\r
2,116853306,162,46.46,80.34,24.22,89.37,13.50,35.19,75.93,48.15,16.05,11.11,2.47,0.62,0.00,3.09,6.79,13.58,7.41,4.32,15.43,1.85,3.09,3.09,0.00,0.00,0.00,0.00,18.52,1.23,0.62,0.00,0.00,10.49,2.47,0.62,0.00,1.85,2.47,3.09,3.09,0.62,1.23,0.00,1.85,0.00,0.62,0.00,0.62,8.02,3.09,4.32,0.00,1.23,0.00,1.23,4.32,1.85,3.70,2.47,3.70,1.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r
3,116959865,94,95.21,77.64,17.77,99.00,11.75,28.72,77.66,43.62,15.96,4.26,0.00,0.00,1.06,7.45,10.64,14.89,2.13,1.06,13.83,2.13,7.45,7.45,0.00,0.00,0.00,0.00,14.89,1.06,2.13,0.00,0.00,11.70,1.06,0.00,1.06,4.26,1.06,4.26,3.19,2.13,1.06,0.00,1.06,0.00,1.06,0.00,0.00,5.32,1.06,2.13,0.00,2.13,1.06,0.00,3.19,1.06,3.19,1.06,6.38,0.00,0.00,0.00,1.06,0.00,1.06,0.00,0.00\r
4,116953174,148,93.60,95.48,2.04,99.00,13.45,37.16,76.35,45.95,13.51,9.46,0.00,4.05,2.03,3.38,8.11,15.54,7.43,4.05,9.46,0.00,6.76,6.08,0.00,0.00,0.00,0.00,12.84,0.68,0.68,0.00,0.68,10.14,0.68,1.35,0.68,0.68,2.03,0.68,0.68,0.00,0.00,0.68,4.05,0.68,0.68,1.35,0.00,13.51,5.41,6.76,0.68,0.00,0.00,0.00,5.41,2.03,4.73,2.03,3.38,0.68,0.00,0.00,2.03,0.00,0.00,0.68,0.00\r
5,116953218,86,99.00,51.14,1.00,17.39,4.53,43.02,51.16,18.60,2.33,0.00,0.00,0.00,0.00,2.33,2.33,8.14,2.33,0.00,9.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.16,0.00,0.00,0.00,0.00,23.26,0.00,0.00,0.00,1.16,1.16,3.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,19.77,18.60,1.16,0.00,0.00,0.00,0.00,0.00,0.00,5.81,1.16,2.33,1.16,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56633,37000048,70,98.84,70.47,4.13,52.93,35.00,38.57,54.29,27.14,4.29,0.00,0.00,0.00,0.00,2.86,2.86,12.86,4.29,0.00,11.43,0.00,1.43,1.43,0.00,0.00,0.00,0.00,5.71,0.00,0.00,0.00,0.00,11.43,1.43,0.00,0.00,0.00,2.86,1.43,0.00,0.00,0.00,0.00,4.29,0.00,0.00,0.00,2.86,7.14,1.43,5.71,0.00,0.00,0.00,0.00,4.29,0.00,0.00,1.43,11.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r
56634,36788963,87,99.00,71.49,64.52,99.00,12.43,27.59,75.86,54.02,9.20,6.90,1.15,0.00,0.00,2.30,16.09,20.69,5.75,2.30,9.20,0.00,8.05,8.05,0.00,0.00,0.00,0.00,9.20,0.00,0.00,0.00,0.00,5.75,0.00,0.00,0.00,1.15,0.00,2.30,6.90,6.90,0.00,0.00,2.30,0.00,0.00,0.00,0.00,6.90,0.00,4.60,2.30,1.15,0.00,1.15,4.60,0.00,2.30,0.00,0.00,1.15,0.00,0.00,1.15,0.00,0.00,0.00,0.00\r
56635,36713355,61,99.00,71.67,54.69,99.00,7.62,39.34,72.13,47.54,6.56,1.64,0.00,1.64,0.00,3.28,16.39,14.75,11.48,3.28,8.20,0.00,8.20,8.20,0.00,0.00,0.00,0.00,1.64,0.00,0.00,0.00,0.00,3.28,0.00,0.00,0.00,1.64,0.00,0.00,1.64,1.64,0.00,0.00,6.56,0.00,0.00,0.00,0.00,4.92,1.64,1.64,0.00,1.64,1.64,1.64,9.84,0.00,1.64,3.28,4.92,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r
56636,36652956,71,74.31,70.35,51.16,99.00,5.46,39.44,64.79,26.76,5.63,2.82,0.00,1.41,0.00,1.41,2.82,9.86,7.04,1.41,15.49,1.41,5.63,5.63,0.00,0.00,0.00,0.00,9.86,1.41,0.00,0.00,0.00,12.68,2.82,0.00,1.41,1.41,0.00,4.23,0.00,0.00,0.00,0.00,1.41,0.00,0.00,0.00,0.00,15.49,4.23,4.23,7.04,0.00,0.00,5.63,2.82,0.00,5.63,1.41,2.82,5.63,0.00,0.00

In [0]:
liwc2_merged = pd.merge(test, liwc2, how="left", left_on="ID", right_on="A")

In [0]:
liwc2_merged

,ID,ScoutID,Typ,Kaufpreis,Provision,Energieeffizienzklasse,Objektzustand,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,Bezugsfrei_ab,Title,Beschreibung,Ausstattung,Lagebeschreibung,Sonstiges,PLZ,Stadt,Ortsteil,ScrapingDate1,ScrapingDate2,ScrapingDate3,ScrapingDate4,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Average_income,Topic,A,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you_total,ipron,article,prep,auxverb,adverb,conj,negate,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achiev,power,reward,risk,focuspast,focuspresent,focusfuture,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu\r
0,115756801,Objekt-Nr.: FB211 | Scout-ID: 115756801,Bungalow,518500.0,Nein,A+,Erstbezug,3.0,160.0,1200.0,2020,nan,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,"3 Zimmer, offene Küche, Diele, Bad, Dusche, Ha...","Das Baugrundstück befindet sich in Dauernheim,...",Kern-Haus baut individuelle und massive Archit...,63691,Ranstadt,Wetteraukreis,2020-04-21,nan,nan,nan,4,nan,nan,nan,nan,nan,nan,0,115756801,260,99.00,75.31,33.30,99.00,15.29,32.31,79.23,45.00,8.46,3.46,0.00,0.00,0.38,3.46,14.23,15.00,6.54,4.62,10.77,0.00,6.15,6.15,0.00,0.00,0.00,0.00,10.38,0.00,1.15,0.00,0.00,13.08,2.31,1.15,1.15,2.69,1.54,3.46,2.31,1.15,0.77,0.38,4.23,0.38,0.77,0.00,1.92,8.08,1.15,2.69,2.31,2.31,0.00,0.38,3.46,1.92,4.23,5.38,6.54,0.38,0.38,0.00,1.15,0.00,0.38,0.00,0.00\r
1,116853306,Objekt-Nr.: 562020 | Scout-ID: 116853306,Einfamilienhaus (freistehend),301667.0,Nein,nan,Erstbezug,4.0,135.0,860.0,nan,nan,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,In dem oben genannten Kaufpreis wurden die Kos...,Die Häuser werden auf realen Grundstücken ange...,Deutschland`s meistverkaufte Ausbauhäuser!\n\n...,52385,Nideggen,Düren,2020-04-21,nan,nan,nan,4,65.04,9945.0,4902.0,5043.0,153.0,nan,12,116853306,162,46.46,80.34,24.22,89.37,13.50,35.19,75.93,48.15,16.05,11.11,2.47,0.62,0.00,3.09,6.79,13.58,7.41,4.32,15.43,1.85,3.09,3.09,0.00,0.00,0.00,0.00,18.52,1.23,0.62,0.00,0.00,10.49,2.47,0.62,0.00,1.85,2.47,3.09,3.09,0.62,1.23,0.00,1.85,0.00,0.62,0.00,0.62,8.02,3.09,4.32,0.00,1.23,0.00,1.23,4.32,1.85,3.70,2.47,3.70,1.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r
2,116959865,Objekt-Nr.: W16091 | Scout-ID: 116959865,Einfamilienhaus (freistehend),225900.0,Nein,A+,Erstbezug,3.0,76.0,612.0,2021,nan,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,"3 Zimmer, offene Küche, Diele mit Garderobe, B...",- Grundstück befindet sich in Elstertrebnitz i...,Kern-Haus\r\n\r\nKern-Haus baut individuelle u...,04523,Pegau,Leipzig,2020-04-21,2020-04-25,2020-04-29,nan,12,48.8,6288.0,3068.0,3220.0,129.0,nan,24,116959865,94,95.21,77.64,17.77,99.00,11.75,28.72,77.66,43.62,15.96,4.26,0.00,0.00,1.06,7.45,10.64,14.89,2.13,1.06,13.83,2.13,7.45,7.45,0.00,0.00,0.00,0.00,14.89,1.06,2.13,0.00,0.00,11.70,1.06,0.00,1.06,4.26,1.06,4.26,3.19,2.13,1.06,0.00,1.06,0.00,1.06,0.00,0.00,5.32,1.06,2.13,0.00,2.13,1.06,0.00,3.19,1.06,3.19,1.06,6.38,0.00,0.00,0.00,1.06,0.00,1.06,0.00,0.00\r
3,116953174,Scout-ID: 116953174,Einfamilienhaus (freistehend),218659.0,Nein,nan,Erstbezug,6.0,144.0,550.0,2020,nan,Das neue LifeStyle 14.03S ! Auch als Stadtvill...,Jeder von uns hat Träume. Ein eigenes Haus zu ...,ZERTIFIZIERTE QUALITÄT IN ALLEN BEREICHEN!\n\n...,"Ich bin Ihr Ansprechpartner in Ihrer Region, w...",Profitieren Sie vom aktuell niedrigen Zinsnive...,52428,52428,Jülich,2020-04-21,2020-04-25,2020-04-29,nan,12,nan,nan,nan,nan,nan,nan,10,116953174,148,93.60,95.48,2.04,99.00,13.45,37.16,76.35,45.95,13.51,9.46,0.00,4.05,2.03,3.38,8.11,15.54,7.43,4.05,9.46,0.00,6.76,6.08,0.00,0.00,0.00,0.00,12.84,0.68,0.68,0.00,0.68,10.14,0.68,1.35,0.68,0.68,2.03,0.68,0.68,0.00,0.00,0.68,4.05,

In [0]:
liwc2_merged = liwc2_merged.drop(['A', 'ScoutID', 'Provision', 'Bezugsfrei_ab', 'Ausstattung', 'Lagebeschreibung', 'Sonstiges', 'Ortsteil', 'ScrapingDate1', 'ScrapingDate2', 'ScrapingDate3', 'ScrapingDate4'], axis=1) 

In [0]:
liwc2_merged

,ID,Typ,Kaufpreis,Energieeffizienzklasse,Objektzustand,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,Title,Beschreibung,PLZ,Stadt,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Average_income,Topic,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you_total,ipron,article,prep,auxverb,adverb,conj,negate,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achiev,power,reward,risk,focuspast,focuspresent,focusfuture,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu\r
0,115756801,Bungalow,518500.0,A+,Erstbezug,3.0,160.0,1200.0,2020,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,63691,Ranstadt,4,nan,nan,nan,nan,nan,nan,0,260,99.00,75.31,33.30,99.00,15.29,32.31,79.23,45.00,8.46,3.46,0.00,0.00,0.38,3.46,14.23,15.00,6.54,4.62,10.77,0.00,6.15,6.15,0.00,0.00,0.00,0.00,10.38,0.00,1.15,0.00,0.00,13.08,2.31,1.15,1.15,2.69,1.54,3.46,2.31,1.15,0.77,0.38,4.23,0.38,0.77,0.00,1.92,8.08,1.15,2.69,2.31,2.31,0.00,0.38,3.46,1.92,4.23,5.38,6.54,0.38,0.38,0.00,1.15,0.00,0.38,0.00,0.00\r
1,116853306,Einfamilienhaus (freistehend),301667.0,nan,Erstbezug,4.0,135.0,860.0,nan,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,52385,Nideggen,4,65.04,9945.0,4902.0,5043.0,153.0,nan,12,162,46.46,80.34,24.22,89.37,13.50,35.19,75.93,48.15,16.05,11.11,2.47,0.62,0.00,3.09,6.79,13.58,7.41,4.32,15.43,1.85,3.09,3.09,0.00,0.00,0.00,0.00,18.52,1.23,0.62,0.00,0.00,10.49,2.47,0.62,0.00,1.85,2.47,3.09,3.09,0.62,1.23,0.00,1.85,0.00,0.62,0.00,0.62,8.02,3.09,4.32,0.00,1.23,0.00,1.23,4.32,1.85,3.70,2.47,3.70,1.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r
2,116959865,Einfamilienhaus (freistehend),225900.0,A+,Erstbezug,3.0,76.0,612.0,2021,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,04523,Pegau,12,48.8,6288.0,3068.0,3220.0,129.0,nan,24,94,95.21,77.64,17.77,99.00,11.75,28.72,77.66,43.62,15.96,4.26,0.00,0.00,1.06,7.45,10.64,14.89,2.13,1.06,13.83,2.13,7.45,7.45,0.00,0.00,0.00,0.00,14.89,1.06,2.13,0.00,0.00,11.70,1.06,0.00,1.06,4.26,1.06,4.26,3.19,2.13,1.06,0.00,1.06,0.00,1.06,0.00,0.00,5.32,1.06,2.13,0.00,2.13,1.06,0.00,3.19,1.06,3.19,1.06,6.38,0.00,0.00,0.00,1.06,0.00,1.06,0.00,0.00\r
3,116953174,Einfamilienhaus (freistehend),218659.0,nan,Erstbezug,6.0,144.0,550.0,2020,Das neue LifeStyle 14.03S ! Auch als Stadtvill...,Jeder von uns hat Träume. Ein eigenes Haus zu ...,52428,52428,12,nan,nan,nan,nan,nan,nan,10,148,93.60,95.48,2.04,99.00,13.45,37.16,76.35,45.95,13.51,9.46,0.00,4.05,2.03,3.38,8.11,15.54,7.43,4.05,9.46,0.00,6.76,6.08,0.00,0.00,0.00,0.00,12.84,0.68,0.68,0.00,0.68,10.14,0.68,1.35,0.68,0.68,2.03,0.68,0.68,0.00,0.00,0.68,4.05,0.68,0.68,1.35,0.00,13.51,5.41,6.76,0.68,0.00,0.00,0.00,5.41,2.03,4.73,2.03,3.38,0.68,0.00,0.00,2.03,0.00,0.00,0.68,0.00\r
4,116953218,Einfamilienhaus (freistehend),287200.0,nan,Erstbezug,4.0,141.0,801.0,nan,"Barriere freier Bungalow in Kehrigk , jetzt ko...",Dieses Angebot beinhaltet:\n\n- Projektiertes ...,15859,Storkow,12,nan,nan,nan,nan,nan,nan,10,86,99.00,51.14,1.00,17.39,4.53,43.02,51.16,18.60,2.33,0.00,0.00,0.00,0.00,2.33,2.33,8.14,2.33,0.00,9.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.16,0.00,0.00,0.00,0.00,23.26,0.00,0.00,0.00,1.16,1.16,3.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,19.77,18.60,1.16,0.00,0.00,0.00,0.00,0.00,0.00,5.81,1.16,2.33,1.16,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53221,37000048,Besondere Immobilie,239000.0,nan,Modernis

In [0]:
liwc3_merged = liwc2_merged.set_index('ID')

In [0]:
liwc3_merged.to_excel('LIWC_Immoscout_complete.xlsx')

In [0]:
liwc3_merged.to_csv('LIWC_Immoscout_complete.csv')